### Initialize ellipsoids with DPD

In [1]:
import gsd, gsd.hoomd 
import hoomd
import mbuild as mb
import numpy as np
import scipy.stats
import unyt as u
import warnings
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer
from flowermd.base.system import System
#from flowermd.library import EllipsoidForcefield, EllipsoidChain
#from flowermd.utils import get_target_box_number_density
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from scipy.spatial.distance import pdist
warnings.filterwarnings('ignore')

/Users/stephaniemccallum/miniforge3/envs/mupt-examples/lib/python3.12/site-packages/gmso/core/element.py:10: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [40]:
class EllipsoidChainRand(Polymer):
    def __init__(
        self,
        lengths,
        num_mols,
        lpar,
        bead_mass,
        density,
        bond_L=0.1,
        name="ellipsoid_chain",
    ):
        self.bead_mass = bead_mass
        self.lpar = lpar
        self.bond_L = bond_L
        self.density = density
        N=lengths*num_mols
        L = np.cbrt(N*4/ self.density) #Changing the number density here! 4 beads per ellipsoid!
        print("L",L)
        self.L = L
        self.box = mb.Box(lengths=np.array([self.L] * 3))
        self.bead_constituents_types = ["X", "A", "T", "T"]
        super(EllipsoidChainRand, self).__init__(
            lengths=lengths, num_mols=num_mols, name=name
        )

    def _build(self, length):
        bead = mb.Compound(name="ellipsoid")
        center = mb.Compound(pos=(0, 0, 0), name="X", mass=self.bead_mass / 4)
        head = mb.Compound(
            pos=(0, 0, self.lpar + (self.bond_L / 2)),
            name="A",
            mass=self.bead_mass / 4,
        )
        tether_head = mb.Compound(
            pos=(0, 0, self.lpar), name="T", mass=self.bead_mass / 4
        )
        tether_tail = mb.Compound(
            pos=(0, 0, -self.lpar), name="T", mass=self.bead_mass / 4
        )
        bead.add([center, head, tether_head, tether_tail])
        bead.add_bond([center, head])

        chain = mb.Compound()
        last_bead = None
        rand_range = self.L/8 #
        print('range',rand_range)
        for i in range(length):
            translate_by = np.random.uniform(low=-rand_range, high=rand_range, size=(3,))
            translate_by /= np.linalg.norm(translate_by)
            translate_by *= self.bond_L
            this_bead = mb.clone(bead)
            if last_bead:
                chain.add_bond([this_bead.children[0], last_bead.children[1]])
                chain.add_bond([this_bead.children[3], last_bead.children[2]])
                this_bead.translate(by=self.pbc(translate_by+last_bead.pos,box=([self.L]*3)))
                print('next bead, pbcs',self.pbc(translate_by+last_bead.pos,box=([self.L]*3)))
            else:
                translate_by = np.random.uniform(low=-rand_range, high=rand_range, size=(3,))
                this_bead.translate(by=translate_by)
                print('first bead', translate_by)
            chain.add(this_bead)
            last_bead = this_bead
        chain.name = f"{self.name}_{length}mer"
        chain.box = self.box
        return chain

    def pbc(self,d,box):
        for i in range(3):
            while d[i] > box[i]/2 or d[i] < -box[i]/2:
                if d[i] < -box[i]/2:
                    d[i] += box[i]
                if d[i] > box[i]/2:
                    d[i] -= box[i]
        return d

class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        lpar,
        lperp,
        A,
        gamma,
        kT,
        r_cut,
        angle_k=None,
        angle_theta0=None,
        bond_k=100,
        bond_r0=0.1,
    ):
        self.epsilon = epsilon
        self.lpar=lpar
        self.lperp=lperp
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        self.angle_k = angle_k
        self.angle_theta0 = angle_theta0
        self.bond_k = bond_k
        self.bond_r0 = bond_r0
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["T-T"] = dict(k=self.bond_k, r0=self.bond_r0)
        bond.params["A-X"] = dict(k=0, r0=0)
        forces.append(bond)
        # Angles
        if all([self.angle_k, self.angle_theta0]):
            angle = hoomd.md.angle.Harmonic()
            angle.params["X-A-X"] = dict(k=self.angle_k, t0=self.angle_theta0)
            angle.params["A-X-A"] = dict(k=0, t0=0)
            forces.append(angle)
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        dpd.params[("X", "X")] = dict(A=self.A, gamma=self.gamma)
        # Add zero pairs
        for pair in [
            ("R", "R"),
            ("T", "T"),
            ("T", "R"),
            ("A", "A"),
            ("A", "X"),
            ("A", "T"),
            ("A", "R"),
            ("X", "R"),
            ("X", "T"),
        ]:
            dpd.params[pair] = dict(A=0,gamma=0.1)
            dpd.params[pair].r_cut = 0.0
        forces.append(dpd)
        return forces

class RandomSystem(System):
    def __init__(self, molecules, base_units=dict()):
        self.L = molecules.L
        print(self.L)
        super(RandomSystem, self).__init__(
            molecules=molecules, base_units=base_units
        )
    def _build_system(self):
        chain = self.all_molecules
        for i in range(len(chain)):
            print(chain[i].pos)
        comp = mb.Compound()
        comp.add(chain)
        comp.box = mb.Box(lengths=np.array([self.L] * 3))
        print(comp.box)
        return comp

In [41]:
ellipsoid_chain = EllipsoidChainRand(lengths=4,num_mols=4,lpar=1.0,bead_mass=1.0,density=0.6)
ff = DPD_FF(epsilon=1.0,lpar=1.0,lperp=0.5,A=100,gamma=1.0,kT=2.0,r_cut=1.94,bond_k=100,bond_r0=1.0,angle_k=30,angle_theta0=1.9)
ff.hoomd_forces
system = RandomSystem(molecules=ellipsoid_chain)
system.to_gsd('test.gsd')
gsd_path=('ellipsoid-test.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot,lpar=1.0,lperp=0.5
)
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.0003,
    gsd_write_freq=int(100),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(100),
    log_file_name='log.txt')

ellipsoid_sim.save_restart_gsd()
ellipsoid_sim.run_NVT(n_steps=1e4, kT=2.0, tau_kt=10*ellipsoid_sim.dt)
ellipsoid_sim.flush_writers()

*Warning*: angle.harmonic: specified K <= 0
*Warning*: angle.harmonic: specified t_0 <= 0


L 4.74252440598675
range 0.5928155507483438
first bead [-0.1468408  -0.37372377 -0.44453616]
next bead, pbcs [-0.219812   -0.31164423 -0.1533772 ]
next bead, pbcs [-0.13737609 -0.3017939   0.05337971]
next bead, pbcs [-0.19172055 -0.28911729  0.23289789]
range 0.5928155507483438
first bead [ 0.46319508  0.01613736 -0.16814188]
next bead, pbcs [0.47406979 0.08548352 0.16558208]
next bead, pbcs [0.44723108 0.17989843 0.40896378]
next bead, pbcs [0.4579632  0.12268915 0.59015007]
range 0.5928155507483438
first bead [-0.06994507  0.16370592 -0.44610637]
next bead, pbcs [-0.06200799  0.23142818 -0.11045791]
next bead, pbcs [-0.07117826  0.15791783  0.08486967]
next bead, pbcs [0.02164243 0.12533617 0.32940429]
range 0.5928155507483438
first bead [-0.54048011 -0.57988247 -0.40786668]
next bead, pbcs [-0.51832588 -0.58382816 -0.04793146]
next bead, pbcs [-0.5960183  -0.63321993  0.17552572]
next bead, pbcs [-0.61148853 -0.59957479  0.34513505]
4.74252440598675
[-0.17393736 -0.3190698   0.1845

In [42]:
snap = ellipsoid_sim.state

In [43]:
traj = gsd.hoomd.open('trajectory.gsd','r')

In [45]:
last_frame = traj[-1]

In [48]:
orientations = last_frame.particles.orientation

TypeError: 'ParticleData' object is not subscriptable

In [56]:
N = 16
print(last_frame.particles.typeid, last_frame.particles.types)
print("orientations of rigid bodies\n",orientations[0:N])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3
 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1 2 2 3 1
 2 2 3 1 2 2] ['R', 'A', 'T', 'X']
orientations of rigid bodies
 [[ 0.95819587 -0.12944622 -0.23277386 -0.10450203]
 [ 0.8327547  -0.5053889   0.2199368   0.05224506]
 [ 0.38545138  0.7716249   0.00990851  0.5058894 ]
 [ 0.42734447  0.38297933 -0.78000134  0.2496025 ]
 [ 0.7453803   0.08721206  0.386733   -0.5359476 ]
 [ 0.88427407 -0.31719303  0.3358522   0.06819997]
 [ 0.86640716  0.28251436 -0.28487873 -0.29726824]
 [ 0.06560332  0.6791976   0.34259656 -0.64576656]
 [ 0.80388564 -0.23035865  0.5167435   0.18351825]
 [ 0.608707   -0.42369705 -0.32450444 -0.58707196]
 [ 0.74577165  0.454552   -0.28320986 -0.39623144]
 [ 0.5953009   0.4191972   0.0622477  -0.6826535 ]
 [ 0.61176294  0.22699863  0.4913547  -0.5768781 ]
 [ 0.663188   -0.54566896  0.05853892 -0.5089207 ]
 [ 0.9357793  -0.14532912 -0.2934678  -0.13066481]
 [ 0.54199445 -0.593

In [60]:
#positions of T particles
positions = last_frame.particles.position
typids = last_frame.particles.typeid
t_idx = np.where(typids == 2)

In [61]:
t_positions = positions[t_idx[0]]

In [62]:
print('ellipsoid anchor positions',t_positions)

ellipsoid anchor positions [[-0.60493124 -1.1861321  -1.9803307 ]
 [ 0.23313096 -1.7795727   1.0459534 ]
 [ 0.14092605  0.9358092  -0.42068696]
 [-0.48607126 -0.7936131  -1.2055264 ]
 [ 1.4872289   0.13862737  0.37355223]
 [-0.08947552  1.3082725   0.75556487]
 [ 0.4697839  -2.2357805  -0.33347455]
 [ 1.4207306  -0.802363    0.68682665]
 [-2.0939555  -0.511006    0.30366609]
 [ 1.6824803   0.5780931  -1.0676605 ]
 [-1.9233603   0.4644333   1.9677925 ]
 [ 1.7177525  -0.7491294   0.82142496]
 [ 1.4825037   0.7263787  -1.3297186 ]
 [-1.9368064   1.3667269   2.0566866 ]
 [ 0.04176214 -1.3706591  -1.8013262 ]
 [ 1.7062726  -0.30747908 -1.4865991 ]
 [ 0.813066    1.6717114  -1.8804272 ]
 [-0.6794447   0.551656    2.1424527 ]
 [ 0.92316043 -2.110072   -0.6858493 ]
 [ 0.7183103   0.8387931  -1.54656   ]
 [-1.2989106  -2.2268639   0.37391913]
 [ 0.26636    -1.3197626  -0.47877955]
 [-1.6493583   0.543793    1.2222016 ]
 [-0.6529171   1.7119614  -0.05939408]
 [ 1.4372889   1.527945    2.1947527 